# Final project (part3)
Team leader: Zhaoliang Zheng

Team member: Anqi Shen, Enbo Yu

## Natural thinking:
### we can use word2vec to vectorize our title and doc2vec to vectorize our text and then put them together. 

In [1]:
import os
ROOTDIR = os.path.abspath(os.path.dirname('__file__'))
print(ROOTDIR)

C:\Users\zzl\data\final project


In [2]:
import pandas as pd
#Load movie reviews dataset
df = pd.read_csv(os.path.join(ROOTDIR, 'fake_or_real_news.csv'))
df.head()

,Unnamed: 0,title,text,label,title_vectors
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,[ 1.1533764e-02 4.2144405e-03 1.9692603e-02 ...
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,[ 0.11267698 0.02518966 -0.00212591 0.021095...
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,[ 0.04253004 0.04300297 0.01848392 0.048672...
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,[ 0.10801624 0.11583211 0.02874823 0.061732...
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,[ 1.69016439e-02 7.13498285e-03 -7.81233795e-...


In [3]:
#in this step, I'm gonna drop the first column which is meaningless here
df.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
#in this step, I'm gonna binrize the label
def binarize(word):
    if word== 'REAL':
        return 0
    else:
        return 1
df.label= df.label.apply(binarize)

In [5]:
df.head()

,title,text,label,title_vectors
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1,[ 1.1533764e-02 4.2144405e-03 1.9692603e-02 ...
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1,[ 0.11267698 0.02518966 -0.00212591 0.021095...
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0,[ 0.04253004 0.04300297 0.01848392 0.048672...
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1,[ 0.10801624 0.11583211 0.02874823 0.061732...
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0,[ 1.69016439e-02 7.13498285e-03 -7.81233795e-...


In [6]:
import nltk
from nltk.corpus import gutenberg
from gensim import models

D:\program files(X86)\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:
from string import punctuation
texts = df.text
titles= df.title
mapping_table = {ord(char): u' ' for char in punctuation}
tokenized_text = [nltk.word_tokenize(review.translate(mapping_table)) for review in texts]
tokenized_title = [nltk.word_tokenize(review.translate(mapping_table)) for review in titles]

In [8]:
from nltk.corpus import stopwords
sw = stopwords.words('english')

In [10]:
from nltk.corpus import stopwords

def clean_advance(tokenized_list, sw):
    new_list = []
    for doc in tokenized_list:
        new_list.append([token.lower() for token in doc if token.lower() not in sw
                        and token.lower() != "“" and token.lower() != "”"
                        and token.lower() != "‘" and token.lower() != "’"
                        and token.lower() != "``" and token.lower() != "''"
                        and token.lower() != "—"  and token.lower() != "–"
                        and token.lower() !="," and token.lower() != "."
                        and token.lower() != ":" and token.lower() != "?"
                        and token.lower() != "(" and token.lower() != ")"
                        and token.lower() != "'s" and token.lower() != "n't"
                        and token.lower() != "'d" and token.lower() != "'ve"
                        and token.lower() != "!" and token.lower() != "'"
                        and token.lower() != "&" and token.lower() != "*"
                        and token.lower() != "..." and token.lower() != "…" 
                        and token.lower() != "#" and token.lower() != "-"
                        and token.lower() != "[" and token.lower() != "]" 
                        and token.lower() != "%" and token.lower() !="|" 
                        and token.lower() != ";"])
    return new_list

# Remove punctuations and stopwords, and lower-case and useless
cleaned_text = clean_advance(tokenized_text, sw)
cleaned_title = clean_advance(tokenized_title, sw)

In [13]:
# Show the most 20 common cleaned text
from collections import Counter

token_counter = Counter(token.lower() for sentence in cleaned_text for token in sentence)
top20 = token_counter.most_common()[:20]
for i, t in enumerate(top20):
    print('{:>2}.{:>11}  freq: {:>7}'.format(i+1, t[0], t[1]))

 1.      trump  freq:   22383
 2.       said  freq:   21200
 3.    clinton  freq:   17461
 4.      would  freq:   12765
 5.        one  freq:   11842
 6.     people  freq:   11712
 7.        new  freq:    9327
 8.      state  freq:    9195
 9.  president  freq:    8825
10.      obama  freq:    8221
11.       also  freq:    8216
12.         us  freq:    7848
13.   campaign  freq:    7792
14.       like  freq:    7219
15.    hillary  freq:    7198
16.       time  freq:    6937
17.      could  freq:    6552
18.       even  freq:    6436
19.     states  freq:    6216
20. republican  freq:    5929


## Word2vec

In [38]:
def w2v_params():
    params = {
        'sg': (0, 1),
        'size': (100, 150,200,250, 300),
        'window': (1,3,4,5, 7),
    }
    return params

In [59]:
import time

start_w2v = time.clock()

from gensim import models
#Word2Vec
def w2v_fit(sg,size,window):
    wvtext = models.Word2Vec(
                       cleaned_title,
                       size = size,
                       window = window,
                       min_count = 1,
                       sg = sg, 
                       alpha = 0.15, 
                       iter=10, batch_words = 10000)
    return wvtext
elapsed_w2v = (time.clock() - start_w2v)

print("Time used:",elapsed_w2v)

Time used: 0.0003552820508048171


In [62]:
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np

y = np.asarray(df['label'])
seed = 42
test_size = 0.15
validation_size = 0.15/(1-test_size)
best_ac=0.0
b_sg=[]
b_size=[]
b_window=[]

for sg,size,window in product( w2v_params()['sg'],w2v_params()['size'], w2v_params()['window']):
        w2vtext = w2v_fit(sg,size,window)
        text_vec = np.zeros((len(df), size))
        for i in range(0, len(df)):
            text_vec[i] = 0
            length = len(cleaned_title[i])
            for word in cleaned_title[i]:
                 text_vec[i] += w2vtext[word]
            if length != 0:
                text_vec[i] = text_vec[i]/length
        X = np.insert(text_vec, 0, 1, axis=1)
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
        x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=validation_size, random_state=seed)
        model = LogisticRegression()
        model = model.fit(x_train, y_train)
        y_pred = model.predict(x_validation)
        accuracy= model.score(x_validation, y_validation)
        if best_ac < accuracy:
            best_ac=accuracy
            b_sg=sg
            b_size=size
            b_window=window
            print('sg: {} , size: {} , window: {} , accuracy: {}'.format(b_sg,b_size,b_window, best_ac))

D:\program files(X86)\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


sg: 0 , size: 100 , window: 1 , accuracy: 0.7697160883280757
sg: 0 , size: 100 , window: 3 , accuracy: 0.7718191377497371
sg: 0 , size: 100 , window: 4 , accuracy: 0.7760252365930599
sg: 0 , size: 100 , window: 5 , accuracy: 0.7917981072555205
sg: 0 , size: 150 , window: 7 , accuracy: 0.7970557308096741
sg: 0 , size: 300 , window: 7 , accuracy: 0.7981072555205048
sg: 1 , size: 150 , window: 4 , accuracy: 0.8012618296529969


In [63]:
import time

start_w2v = time.clock()

from gensim import models
#Word2Vec
wvtext = models.Word2Vec(
                       cleaned_title,
                       size = 150,
                       window = 4,
                       min_count = 1,
                       sg = 1, 
                       alpha = 0.15, 
                       iter=10, batch_words = 10000)

elapsed_w2v = (time.clock() - start_w2v)

print("Time used:",elapsed_w2v)

Time used: 1.3659364102559266


In [64]:
from numpy import *
import numpy as np

text_vec = np.zeros((len(df), 150))
for i in range(0, len(df)):
    text_vec[i] = 0
    length = len(cleaned_title[i])
    for word in cleaned_title[i]:
        text_vec[i] += wvtext[word]
    if length != 0:
        text_vec[i] = text_vec[i]/length #Mean vector value for each text
text_vec.shape

D:\program files(X86)\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


(6335, 150)

## doc2vec

In [65]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk.corpus import reuters

class TagDocIterator:
    def __init__(self, doc_list, idx_list):
        self.doc_list = doc_list
        self.idx_list = idx_list
        
    def __iter__(self):
        for doc, idx, in zip(self.doc_list, self.idx_list):
            tag = [idx]
            yield TaggedDocument(words=doc, tags=tag)

In [66]:
def d2v_fit(dm,size,window,mean,min_count):
  doc2vec = Doc2Vec(size=size, 
                    window=window,
                    min_count=min_count,
                    dm = dm,
                    mean=mean,
                    alpha = 0.05,
                    iter=10)
  # Build the word2vec model from the corpus
  doc2vec.build_vocab(TagDocIterator(cleaned_text, df.index))
  doc2vec.train(TagDocIterator(cleaned_text, df.index), epochs = 5, total_examples = doc2vec.corpus_count)
  return doc2vec.docvecs

In [106]:
docvecs = d2v_fit(0,150,8,1,10) 

D:\program files(X86)\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:355: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
D:\program files(X86)\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:359: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [107]:
import numpy as np
x = np.empty([df.shape[0],150])
for i in range(df.shape[0]):
            x[i] = docvecs[i]

In [122]:
x01 = np.hstack((text_vec, x))

In [123]:
import numpy as np

X = np.insert(x01, 0, 1, axis=1)
y = np.asarray(df['label'])

In [127]:
#forming training sets and test sets
from sklearn.model_selection import train_test_split

seed = 50
test_size = 0.15 #
validation_size = 0.15/(1-test_size)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=validation_size, random_state=seed)
print(y_train.shape)
print(y_test.shape)
print(y_validation.shape)

(4433,)
(951,)
(951,)


In [128]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model = model.fit(x_train, y_train)

y_pred = model.predict(x_validation)
print(classification_report(y_validation, y_pred, target_names=target_names))
print(model.score(x_validation, y_validation))

             precision    recall  f1-score   support

       fake       0.94      0.94      0.94       488
       real       0.94      0.94      0.94       463

avg / total       0.94      0.94      0.94       951

0.9411146161934806


In [151]:
y_tpred = model.predict(x_test)
print(classification_report(y_test, y_tpred, target_names=target_names))
print(model.score(x_test, y_test))

             precision    recall  f1-score   support

       fake       0.93      0.93      0.93       478
       real       0.93      0.93      0.93       473

avg / total       0.93      0.93      0.93       951

0.9327024185068349


In [152]:
from sklearn.svm import SVC
svc = SVC()
target_names = ['fake', 'real']

svc.fit(x_train, y_train)

y_pred = svc.predict(x_validation)
print("SVM:")
print(classification_report(y_validation, y_pred, target_names=target_names))
print(svc.score(x_validation, y_validation))

SVM:
             precision    recall  f1-score   support

       fake       0.94      0.93      0.94       488
       real       0.93      0.94      0.94       463

avg / total       0.94      0.94      0.94       951

0.9369085173501577


In [131]:
from xgboost import XGBClassifier
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
xgb1.fit(x_train, y_train)
y_vpred = xgb1.predict(x_validation)

target_names = ['fake', 'real']
print("XGBoost:")
print(classification_report(y_validation, y_vpred, target_names=target_names))
print(xgb1.score(x_validation, y_validation))

XGBoost:
             precision    recall  f1-score   support

       fake       0.94      0.95      0.95       488
       real       0.95      0.94      0.94       463

avg / total       0.94      0.94      0.94       951

0.943217665615142


D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [134]:
import numpy as np
from time import time
from operator import itemgetter
from scipy.stats import randint as sp_randint
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(x_validation,y_validation)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=2, missing=None, n_estimators=140,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'max_depth': range(3, 10, 2), 'min_child_weight': range(1, 6, 2)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [135]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.96755, std: 0.01108, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.96568, std: 0.01079, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.96308, std: 0.01030, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.96696, std: 0.01096, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.96393, std: 0.00976, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.96374, std: 0.01008, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.96419, std: 0.01043, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.96534, std: 0.00936, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.96457, std: 0.00880, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.96554, std: 0.00888, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.96427, std: 0.00963, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.96457, std: 0.00880, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 3, 'min_child_weight': 1

In [138]:
param_test2 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=3,
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(x_validation,y_validation)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=140,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'gamma': [0.0, 0.1, 0.2, 0.3, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [139]:
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.96755, std: 0.01108, params: {'gamma': 0.0},
  mean: 0.96714, std: 0.01108, params: {'gamma': 0.1},
  mean: 0.96798, std: 0.01174, params: {'gamma': 0.2},
  mean: 0.96827, std: 0.01150, params: {'gamma': 0.3},
  mean: 0.96735, std: 0.01158, params: {'gamma': 0.4}],
 {'gamma': 0.3},
 0.9682656174339141)

In [140]:
param_test3 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=3,
                                        min_child_weight=1, gamma=0.3, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(x_validation,y_validation)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=140, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.8),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [141]:
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.97099, std: 0.00989, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
  mean: 0.97162, std: 0.00684, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
  mean: 0.96960, std: 0.01049, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
  mean: 0.97040, std: 0.00929, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
  mean: 0.97101, std: 0.00928, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
  mean: 0.97072, std: 0.00934, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
  mean: 0.96751, std: 0.01042, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
  mean: 0.96953, std: 0.00960, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
  mean: 0.96973, std: 0.01118, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
  mean: 0.97071, std: 0.00861, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: 0.96827, std: 0.01150, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: 0.96581, std: 0.01119, params: {'colsample_bytree': 0.8, 'subsample'

In [143]:
param_test4 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=3,
                                        min_child_weight=1, gamma=0.3, subsample=0.7, colsample_bytree=0.6,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(x_validation,y_validation)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=140, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.7),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'reg_alpha': [1e-05, 0.01, 0.1, 1, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [144]:
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.97162, std: 0.00684, params: {'reg_alpha': 1e-05},
  mean: 0.97186, std: 0.00730, params: {'reg_alpha': 0.01},
  mean: 0.97218, std: 0.00806, params: {'reg_alpha': 0.1},
  mean: 0.96976, std: 0.01130, params: {'reg_alpha': 1},
  mean: 0.50000, std: 0.00000, params: {'reg_alpha': 100}],
 {'reg_alpha': 0.1},
 0.9721833763218566)

In [145]:
param_test5 = {
    'reg_alpha':[0.01, 0.05, 0.1, 0.15, 0.2]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=3,
                                        min_child_weight=1, gamma=0.3, subsample=0.7, colsample_bytree=0.6,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch5.fit(x_validation,y_validation)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=0.3, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=140, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.7),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'reg_alpha': [0.01, 0.05, 0.1, 0.15, 0.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [146]:
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.97186, std: 0.00730, params: {'reg_alpha': 0.01},
  mean: 0.97292, std: 0.00828, params: {'reg_alpha': 0.05},
  mean: 0.97218, std: 0.00806, params: {'reg_alpha': 0.1},
  mean: 0.97276, std: 0.00765, params: {'reg_alpha': 0.15},
  mean: 0.97189, std: 0.00802, params: {'reg_alpha': 0.2}],
 {'reg_alpha': 0.05},
 0.9729220205599338)

In [147]:
xgb2 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=3,
        min_child_weight=1,
        gamma=0.3,
        subsample=0.7,
        colsample_bytree=0.6,
        reg_alpha=0.05,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
xgb2.fit(x_train, y_train)
y_vpred = xgb2.predict(x_validation)

target_names = ['fake', 'real']
print("XGBoost:")
print(classification_report(y_validation, y_vpred, target_names=target_names))
print(xgb2.score(x_validation, y_validation))

XGBoost:
             precision    recall  f1-score   support

       fake       0.94      0.95      0.94       488
       real       0.95      0.94      0.94       463

avg / total       0.94      0.94      0.94       951

0.9421661409043113


D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\program files(X86)\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## Final result:

![](final.png)

## Conclusion:
The last way, title(word2vec)+test(doc2vec) is slightly better.

## Inspiration：
No matter how hard the question is, no matter how many difficulties that you're going to face, there is always a way out of this. What really matter is you don't want to do, you don't want to think, you don't want to use your mind.